## Time Series Forecasting of Stock Market

In [1]:
# Importing important libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error
from pmdarima.arima import auto_arima

c:\Users\Priyangshu\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [147]:
data=pd.read_csv("TSLA.csv")
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-05-21,39.551998,41.480000,39.208000,41.015999,90019500,0,0.0
1,2019-05-22,39.820000,40.787998,38.355999,38.546001,93426000,0,0.0
2,2019-05-23,38.868000,39.894001,37.243999,39.098000,132735500,0,0.0
3,2019-05-24,39.966000,39.995998,37.750000,38.125999,70683000,0,0.0
4,2019-05-28,38.240002,39.000000,37.570000,37.740002,51564500,0,0.0


## EDA & Feature Engineering


- **Univariate time series**: A univariate time series is a sequence of measurements of the same variable collected over time.

In [148]:
# univarite timeseries
stock_data=data[["Date","Close"]]

In [149]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    758 non-null    object 
 1   Close   758 non-null    float64
dtypes: float64(1), object(1)
memory usage: 12.0+ KB


## Observation:
**From the previous step, we observed that:**

- There are no missing (null) values in both the Date and Close columns.
- The Date column is currently of data type object (string).
- The Close column is of type float64, which is appropriate for numerical analysis.

**Next Step:**

- To make the Date column usable for time series analysis and suitable for algorithmic processing, we need to convert the Date column from its current object data type to a datetime format. This will allow us to perform time-based operations and manipulations on the dataset more efficiently.



In [150]:
stock_data["Date"]=pd.to_datetime(stock_data["Date"])

In [151]:
print(stock_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    758 non-null    datetime64[ns]
 1   Close   758 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 12.0 KB
None


In [54]:
stock_data=stock_data.set_index("Date")

In [55]:
stock_data.describe()

,Close
count,758.000000
mean,485.531513
std,353.160353
min,35.793999
25%,112.323500
50%,488.125000
75%,762.142502
max,1229.910034


### Why Convert the `Date` Column to an Index?

While it is possible to retrieve and visualize data without converting the `Date` column to an index, converting it provides several key benefits, especially for time series analysis:

#### Optimized for Time-Based Operations:
Converting the `Date` column to a `DatetimeIndex` makes it easier and more efficient to perform time-based operations such as resampling, shifting, and rolling window calculations. Libraries like `scipy` and `statsmodels` are optimized for handling time series data when the `Date` is set as an index.

#### Required for Time Series Algorithms:
Many time series forecasting models, including those in `statsmodels`, expect the `Date` column to be in the index for proper functioning. This ensures that time-based dependencies in the data (like trend, seasonality) are captured correctly by the algorithms.

#### Improved Data Retrieval:
Setting the `Date` as the index allows for more intuitive and efficient data retrieval using time slices (e.g., filtering by a specific date range or frequency).

#### Simplified Visualization:
When the `Date` is the index, libraries like `Matplotlib`, `Seaborn`, and `Plotly` automatically treat it as the x-axis in time series plots, simplifying the visualization process.

---

In summary, while direct retrieval and visualization are possible without this step, converting the `Date` column to an index improves performance, makes time series operations easier, and aligns with the requirements of specialized time series libraries.


In [59]:
# EDA: Composition
# Plotting closing price over time
data_1=stock_data.reset_index()
fig = px.line(data_1, x='Date', y='Close', title='Closing Price Over Time')
fig.update_layout(title_text='Closing Price Over Time', title_x=0.5, template='plotly_dark')
fig.show()

### Closing Price Over Time

The line plot visualizes the **closing price** of the stock over time. It helps in identifying the trends, patterns, or fluctuations in the stock price across the entire time period. The x-axis represents the **Date**, while the y-axis represents the **Close** price. A dark theme is used to enhance readability and emphasize the movement of the closing prices.


In [60]:
stock_data.head()

,Close
Date,
2019-05-21,41.015999
2019-05-22,38.546001
2019-05-23,39.098000
2019-05-24,38.125999
2019-05-28,37.740002


In [62]:
data_1.head()

,Date,Close
0,2019-05-21,41.015999
1,2019-05-22,38.546001
2,2019-05-23,39.098000
3,2019-05-24,38.125999
4,2019-05-28,37.740002


In [138]:
# EDA: Distribution


# Create the subplot
fig = make_subplots(rows=1, cols=1, subplot_titles=(""))

# Add the histogram for 'Close' prices
fig.add_trace(go.Histogram(x=stock_data['Close'], name='Close', histnorm='probability density'), row=1, col=1)

# Compute the KDE for 'Close' prices
kde = gaussian_kde(stock_data['Close'])
x_vals = np.linspace(stock_data['Close'].min(), stock_data['Close'].max(), 1000)
kde_vals = kde(x_vals)

# Add the KDE smoothing curve with sky blue color
fig.add_trace(go.Scatter(x=x_vals, y=kde_vals, mode='lines', name='KDE Smoothing',
                         line=dict(color='skyblue', width=2)), row=1, col=1)

# Update the layout
fig.update_layout(title_text='Distribution of Features with KDE', title_x=0.5, template='plotly_dark')

# Show the figure
fig.show()

### Distribution of Closing Prices with KDE

The histogram represents the **distribution of closing prices** of the stock. The y-axis shows the **probability density**, giving an understanding of how often certain price ranges occur. Overlaid on the histogram is a **Kernel Density Estimate (KDE) smoothing curve** in sky blue, which helps visualize the underlying distribution pattern more smoothly. This allows us to identify the most frequent price ranges and the overall shape of the distribution, providing insights into stock price behavior. The plot is displayed using a dark theme for clarity.


In [67]:
#Test for staionarity

adft=adfuller(data_1["Close"])
def test_stationarity(timeseries):
    # Calculate rolling mean and standard deviation with a window size of 183
    rollmean = data_1['Close'].rolling(183).mean()
    rolstd = data_1['Close'].rolling(183).std()

    # Plot the closing price over time using Plotly Express
    fig = px.line(data_1, x='Date', y='Close', title='Closing Price Over Time')

    # Add the rolling mean (with Plotly Graph Objects)
    fig.add_trace(go.Scatter(x=data_1['Date'], y=rollmean, mode='lines', name='Rolling Mean (183 days)',
                         line=dict(color='orange', width=2)))

    # Add the rolling standard deviation (with Plotly Graph Objects)
    fig.add_trace(go.Scatter(x=data_1['Date'], y=rolstd, mode='lines', name='Rolling Std (183 days)',
                         line=dict(color='green', width=2)))

    # Update layout
    fig.update_layout(title_text='Closing Price Over Time with Rolling Mean and Std', title_x=0.5, template='plotly_dark')

    # Show the figure
    fig.show()

    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    # hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)

    # Extract the p-value
    p_value = output[1]

    '''# Print the ADF test result
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {p_value}')
    print(f'Critical Values: {result[4]}')'''

    # Decision based on p-value
    if p_value <= 0.05:
        print("p-value is {} <= 0.05: We reject the null hypothesis. The time series is stationary.".format(p_value))
    else:
        print("p-value is {} > 0.05: We fail to reject the null hypothesis. The time series is non-stationary.".format(p_value))

### Function: `test_stationarity()`

This function is designed to check the stationarity of a time series using the **Augmented Dickey-Fuller (ADF) test**. It also visualizes the time series with rolling mean and standard deviation to give an idea of the stability of the data over time.

#### Key Steps:
1. **Rolling Mean & Standard Deviation Calculation**:
   - The function calculates the **rolling mean** and **rolling standard deviation** over a window size of 183 days (approximately 6 months). These metrics help in visualizing the long-term stability of the time series.

2. **Plotting with Plotly**:
   - A line plot is created to visualize the **closing price over time** using Plotly Express.
   - Two additional traces are added:
     - **Rolling Mean**: Plotted in orange to show the smoothed average of the closing prices over the 183-day window.
     - **Rolling Standard Deviation**: Plotted in green to show how the volatility of the closing prices fluctuates over time.
   - The layout uses a dark theme to enhance readability.

3. **Augmented Dickey-Fuller Test**:
   - The **ADF test** is performed on the time series (`Close` prices) to determine if it is stationary. 
   - The ADF test outputs several statistics:
     - **Test Statistic**: Determines how extreme the result is.
     - **p-value**: The probability of observing the test statistic under the null hypothesis (i.e., the series is non-stationary).
     - **No. of lags used**: The number of lagged differences included in the test.
     - **Number of observations**: The sample size used for the test.
     - **Critical values**: The critical values for the test statistic at different significance levels (1%, 5%, 10%).

4. **Decision Based on p-value**:
   - If the **p-value** is less than or equal to 0.05, the null hypothesis (that the series is non-stationary) is rejected, meaning the time series is stationary.
   - If the **p-value** is greater than 0.05, the null hypothesis is not rejected, indicating the time series is non-stationary.

The function helps determine whether the time series data is stationary or non-stationary, which is a critical step in time series modeling for forecasting.


In [68]:
test_stationarity(data_1.Close)

Results of dickey fuller test
Test Statistics                 -1.363009
p-value                          0.599876
No. of lags used                 9.000000
Number of observations used    748.000000
critical value (1%)             -3.439123
critical value (5%)             -2.865412
critical value (10%)            -2.568832
dtype: float64
p-value is 0.5998762543050695 > 0.05: We fail to reject the null hypothesis. The time series is non-stationary.


### Results of the Dickey-Fuller Test

The **Augmented Dickey-Fuller (ADF) test** was applied to the time series of closing prices, and the following results were obtained:

- **Test Statistic**: -1.363
- **p-value**: 0.5999
- **Number of Lags Used**: 9
- **Number of Observations Used**: 748
- **Critical Values**:
  - 1% level: -3.4391
  - 5% level: -2.8654
  - 10% level: -2.5688

### Interpretation:
Since the **p-value** (0.5999) is **greater than 0.05**, we fail to reject the null hypothesis, meaning the time series is **non-stationary**. This indicates that the statistical properties of the time series (mean, variance, etc.) are not constant over time, which can affect forecasting performance. 

To model this data effectively, additional techniques like **differencing** or **transformations** may be required to make the series stationary before proceeding with time series forecasting models such as ARIMA.


In [152]:
# Assume stock_data['Close'] contains the data
data = data_1['Close'].dropna()

# Create the box plot
box_trace = go.Box(
    y=data, 
    name="Closing Price", 
    boxmean=True, 
    marker_color='lightblue', 
    hoverinfo='y'  # Display the y-values when hovering
)

# Create the figure and add the box plot trace
fig = go.Figure()

# Add the box plot trace to the figure
fig.add_trace(box_trace)

# Update layout to increase plot size and enable hover interaction
fig.update_layout(
    title_text='Box Plot of Closing Prices', 
    title_x=0.5, 
    template='plotly_dark',
    width=1000,  # Increase the width of the plot
    height=600,  # Increase the height of the plot
    hovermode='x unified',  # Show values when hovering along the x-axis
)

# Show the figure
fig.show()

## Box Plot of Closing Prices

This box plot visualizes the distribution of closing prices for a given stock. 

### Key Features:

- **Median (Central Line)**: The central line within the box represents the median of the closing prices, which is the middle value when the data is sorted in ascending order.

- **Interquartile Range (IQR)**: The box itself spans from the 25th percentile (Q1) to the 75th percentile (Q3) of the data. This range contains the central 50% of the closing prices.

- **Box Mean**: The box plot includes a marker for the mean of the data, indicated by a line within the box. This provides an average value of the closing prices.

- **No Outliers**: In this box plot, there are no outliers present. Outliers would typically be shown as individual points outside the "whiskers" of the box. The absence of these points suggests that all closing prices fall within the expected range, as determined by the box plot.

- **Hover Information**: When hovering over the plot, the exact values of the closing prices are displayed, offering more precise information about the data points.

### Visual Details:

- **Color**: The box is colored light blue for better visual distinction.
- **Template**: The plot uses a dark theme, making it easier to view in low-light environments.

The plot provides a clear and concise representation of the distribution of closing prices, helping to quickly understand the central tendency and variability of the stock prices without any apparent anomalies.


In [71]:
from statsmodels.tsa.seasonal import seasonal_decompose
  # Ensure 'Date' is set as the index

# Additive decomposition
result_additive = seasonal_decompose(stock_data["Close"], model='additive', period=182)

# Multiplicative decomposition
result_multiplicative = seasonal_decompose(stock_data["Close"], model='multiplicative', period=182)

# Function to plot decomposition results
def plot_decomposition(result, title):
    fig = go.Figure()

    # Trend component
    fig.add_trace(go.Scatter(
        x=stock_data.index, 
        y=result.trend, 
        mode='lines', 
        name='Trend', 
        line=dict(color='blue', width=2),
        hoverinfo='x+y'
    ))

    # Seasonal component
    fig.add_trace(go.Scatter(
        x=stock_data.index, 
        y=result.seasonal, 
        mode='lines', 
        name='Seasonal', 
        line=dict(color='green', width=2),
        hoverinfo='x+y'
    ))

    # Residual component
    fig.add_trace(go.Scatter(
        x=stock_data.index, 
        y=result.resid, 
        mode='lines', 
        name='Residual', 
        line=dict(color='red', width=2),
        hoverinfo='x+y'
    ))

    # Update layout to increase plot size and enable hover interaction
    fig.update_layout(
        title_text=title, 
        title_x=0.5, 
        template='plotly_dark',
        width=1000,  # Increase the width of the plot
        height=600,  # Increase the height of the plot
        hovermode='x unified'  # Show values when hovering along the x-axis
    )
    
    fig.show()

# Plot the additive decomposition
plot_decomposition(result_additive, "Additive Time Series Decomposition")

# Plot the multiplicative decomposition
plot_decomposition(result_multiplicative, "Multiplicative Time Series Decomposition")

## Time Series Decomposition

The provided code performs time series decomposition to analyze the underlying components of the closing prices of a stock. Time series decomposition helps in understanding different aspects of the time series data, such as trends, seasonal effects, and residual noise.

### Decomposition Models:

- **Additive Decomposition**: This model assumes that the time series can be expressed as the sum of the trend, seasonal, and residual components. The formula for an additive model is:
  
  \[
  Y_t = T_t + S_t + R_t
  \]
  
  where:
  - \(Y_t\) = Observed value at time \(t\)
  - \(T_t\) = Trend component at time \(t\)
  - \(S_t\) = Seasonal component at time \(t\)
  - \(R_t\) = Residual component at time \(t\)

  This model is suitable when the seasonal fluctuations are roughly constant in magnitude over time.

- **Multiplicative Decomposition**: This model assumes that the time series can be expressed as the product of the trend, seasonal, and residual components. The formula for a multiplicative model is:
  
  \[
  Y_t = T_t \times S_t \times R_t
  \]
  
  where:
  - \(Y_t\) = Observed value at time \(t\)
  - \(T_t\) = Trend component at time \(t\)
  - \(S_t\) = Seasonal component at time \(t\)
  - \(R_t\) = Residual component at time \(t\)

  This model is appropriate when the seasonal fluctuations increase or decrease proportionally to the level of the time series.

### Components Analyzed:

1. **Trend Component**: Represents the long-term movement or direction in the time series data. This component shows how the stock's closing prices are trending over time.

2. **Seasonal Component**: Represents the repeating patterns or cycles in the data at specific intervals (e.g., annual or quarterly). This component highlights periodic fluctuations in the closing prices.

3. **Residual Component**: Represents the noise or random variations in the time series data that cannot be explained by the trend or seasonal components. This component shows the irregularities and deviations from the expected pattern.

### Plot Details:

- **Additive Decomposition Plot**: Displays the trend, seasonal, and residual components of the time series using an additive model. 

- **Multiplicative Decomposition Plot**: Displays the trend, seasonal, and residual components of the time series using a multiplicative model.

#### Visual Details:

- **Trend Component**: Shown in blue, illustrating the general direction of the closing prices over time.
  
- **Seasonal Component**: Shown in green, highlighting periodic patterns in the closing prices.

- **Residual Component**: Shown in red, depicting the irregular variations in the data.

- **Hover Information**: Provides exact values for each component when hovering over the plot, making it easier to analyze specific data points.

- **Plot Layout**: The plots use a dark theme for better visibility, with increased width and height for clarity. Hover interaction is enabled along the x-axis to enhance data exploration.

These plots provide a comprehensive view of the stock's closing prices, helping to understand its underlying structure and variations.


In [72]:
# ploting ACF, PACF


# Ensure 'stock_data' contains the 'Close' column
data = data_1['Close'].dropna()

# Calculate ACF and PACF
acf_values, confint_acf = acf(data, nlags=40, alpha=0.05, fft=True)
pacf_values, confint_pacf = pacf(data, nlags=40, alpha=0.05)

# Create the ACF plot
fig_acf = go.Figure()

fig_acf.add_trace(go.Scatter(
    x=np.arange(len(acf_values)),
    y=acf_values,
    mode='markers+lines',
    name='ACF',
    marker=dict(color='blue'),
    line=dict(color='blue'),
    hoverinfo='x+y'
))

# Add ACF confidence intervals
for i in range(len(confint_acf)):
    fig_acf.add_trace(go.Scatter(
        x=[i, i],
        y=confint_acf[i],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        showlegend=False,
        hoverinfo='x+y'
    ))

fig_acf.update_layout(
    title_text='Autocorrelation Function (ACF)',
    title_x=0.5,
    template='plotly_dark',
    width=1000,
    height=600,
    xaxis_title='Lag',
    yaxis_title='ACF',
    hovermode='x unified'
)

# Create the PACF plot
fig_pacf = go.Figure()

fig_pacf.add_trace(go.Scatter(
    x=np.arange(len(pacf_values)),
    y=pacf_values,
    mode='markers+lines',
    name='PACF',
    marker=dict(color='green'),
    line=dict(color='green'),
    hoverinfo='x+y'
))

# Add PACF confidence intervals
for i in range(len(confint_pacf)):
    fig_pacf.add_trace(go.Scatter(
        x=[i, i],
        y=confint_pacf[i],
        mode='lines',
        line=dict(color='green', dash='dash'),
        showlegend=False,
        hoverinfo='x+y'
    ))

fig_pacf.update_layout(
    title_text='Partial Autocorrelation Function (PACF)',
    title_x=0.5,
    template='plotly_dark',
    width=1000,
    height=600,
    xaxis_title='Lag',
    yaxis_title='PACF',
    hovermode='x unified'
)

# Show the figures
fig_acf.show()
fig_pacf.show()


## Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF)

The provided code calculates and visualizes the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) for the closing prices of a stock. These functions are useful for understanding the temporal dependencies in the time series data.

### Autocorrelation Function (ACF)

**Autocorrelation** measures the correlation of a time series with its past values. It helps in identifying repeating patterns or periodicity in the data.

#### Formula:
The autocorrelation at lag \( k \) is calculated as:

\[
\rho_k = \frac{\text{Cov}(Y_t, Y_{t-k})}{\text{Var}(Y_t)}
\]

where:
- \(\rho_k\) = Autocorrelation at lag \( k \)
- \(\text{Cov}(Y_t, Y_{t-k})\) = Covariance between \(Y_t\) and \(Y_{t-k}\)
- \(\text{Var}(Y_t)\) = Variance of \(Y_t\)

**Plot Details:**

- **Markers and Lines**: Shows the autocorrelation values for various lags. The markers are blue, and the line connecting them is also blue.
  
- **Confidence Intervals**: Dashed lines represent the confidence intervals for the autocorrelation values. These intervals help in determining the significance of the autocorrelation values.

#### Visualization:
The ACF plot visualizes how the closing prices of the stock are correlated with their own past values. 

### Partial Autocorrelation Function (PACF)

**Partial Autocorrelation** measures the correlation of a time series with its past values after removing the effects of intermediate lags. It helps in identifying the direct relationship between a time series and its lags.

#### Formula:
The partial autocorrelation at lag \( k \) is calculated as:

\[
\phi_{kk} = \rho_k - \sum_{i=1}^{k-1} \phi_{ki} \rho_{i}
\]

where:
- \(\phi_{kk}\) = Partial autocorrelation at lag \( k \)
- \(\rho_k\) = Autocorrelation at lag \( k \)
- \(\phi_{ki}\) = Partial autocorrelation at lag \( i \)
- \(\rho_{i}\) = Autocorrelation at lag \( i \)

**Plot Details:**

- **Markers and Lines**: Shows the partial autocorrelation values for various lags. The markers are green, and the line connecting them is also green.

- **Confidence Intervals**: Dashed lines represent the confidence intervals for the partial autocorrelation values. These intervals help in determining the significance of the partial autocorrelation values.

#### Visualization:
The PACF plot visualizes the direct correlation between the closing prices of the stock and their past values, accounting for the influence of intermediate lags.

### Plot Layout:

- **Theme**: Both plots use a dark theme for better visibility.

- **Dimensions**: The plots are sized with increased width (1000) and height (600) for clarity.

- **Hover Information**: Provides exact values for ACF and PACF components when hovering over the plots.

- **Axis Titles**: Labeled with 'Lag' for the x-axis and 'ACF' or 'PACF' for the y-axis to clearly indicate the data being represented.

These plots help in understanding the autocorrelation and partial autocorrelation structures in the stock's closing prices, aiding in model selection for time series forecasting.


In [73]:
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller
from plotly.subplots import make_subplots

def plot_stationarity(timeseries):
    # Determine rolling statistics
    rolmean = timeseries.rolling(12).mean()  # rolling mean
    rolstd = timeseries.rolling(12).std()    # rolling standard deviation

    # Plot rolling statistics with Plotly
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

    fig.add_trace(go.Scatter(x=timeseries.index, y=timeseries, mode='lines', name='Original', line=dict(color='blue', width=3)), row=1, col=1)
    fig.add_trace(go.Scatter(x=rolmean.index, y=rolmean, mode='lines', name='Rolling Mean', line=dict(color='red', width=3, dash='dash')), row=1, col=1)
    fig.add_trace(go.Scatter(x=rolstd.index, y=rolstd, mode='lines', name='Rolling Std', line=dict(color='yellow', width=3, dash='dot')), row=1, col=1)

    fig.update_layout(title='Rolling Mean and Standard Deviation',
                      title_font_size=25,
                      xaxis_title='Date',
                      yaxis_title='Value',
                      xaxis_title_font_size=15,
                      yaxis_title_font_size=15,
                      legend=dict(x=1.05, y=0.5, font=dict(size=15), bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)'),template='plotly_dark')

    fig.show()

def test_stationarity(timeseries):
    adft = adfuller(timeseries, autolag='AIC')
    output = pd.Series(adft[0:4], index=['Test Statistics', 'p-value', 'No. of lags used', 'Number of observations used'])
    for key, value in adft[4].items():
        output[f'Critical Value ({key})'] = value
    
    '''# Print the ADF test result
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {p_value}')
    print(f'Critical Values: {result[4]}')'''

    # Decision based on p-value
    p_value = output[1]
    
    if p_value <= 0.05:
        print("p-value is {} <= 0.05: We reject the null hypothesis. The time series is stationary.".format(p_value))
    else:
        print("p-value is {} > 0.05: We fail to reject the null hypothesis. The time series is non-stationary.".format(p_value))
    return output

df_close = stock_data['Close']
df_close = df_close.diff().dropna()

# Plot and test
plot_stationarity(df_close)
print("Results of Dickey-Fuller test:")
print(test_stationarity(df_close))


Results of Dickey-Fuller test:
p-value is 3.498786392123736e-13 <= 0.05: We reject the null hypothesis. The time series is stationary.
Test Statistics               -8.324564e+00
p-value                        3.498786e-13
No. of lags used               8.000000e+00
Number of observations used    7.480000e+02
Critical Value (1%)           -3.439123e+00
Critical Value (5%)           -2.865412e+00
Critical Value (10%)          -2.568832e+00
dtype: float64


## Stationarity Analysis:

The following code performs stationarity analysis on the stock's closing prices by first transforming the data and then using the previously defined functions to plot and test for stationarity.

### Code Details

1. **Prepare the Data**:
   - The closing prices are extracted from the `stock_data` DataFrame.
   - The `.diff()` method is used to compute the first difference of the time series to help achieve stationarity by removing trends.
   - `.dropna()` is applied to remove any missing values that result from differencing.

   ```python
   df_close = stock_data['Close']
   df_close = df_close.diff().dropna()

## Results of Stationarity Analysis

### Plot of Rolling Statistics

The rolling mean and standard deviation plots were generated to visually inspect the stationarity of the time series. The visual inspection of the rolling mean (red dashed line) and rolling standard deviation (yellow dotted line) suggests whether these statistics remain constant over time.

### Dickey-Fuller Test Results

The Augmented Dickey-Fuller (ADF) test was performed on the differenced closing prices to test for stationarity. Here are the results:

- **Test Statistic**: `-8.324564e+00`
- **p-value**: `3.498786e-13`
- **Number of Lags Used**: `8`
- **Number of Observations Used**: `748`

**Critical Values**:
- **1% Critical Value**: `-3.439123e+00`
- **5% Critical Value**: `-2.865412e+00`
- **10% Critical Value**: `-2.568832e+00`

#### Interpretation:

- **p-value**: The p-value is `3.498786e-13`, which is significantly less than the 0.05 threshold. Therefore, we reject the null hypothesis that the time series has a unit root.
- **Test Statistic**: The test statistic of `-8.324564e+00` is more negative than the critical values at the 1%, 5%, and 10% levels, providing strong evidence against the null hypothesis.

**Conclusion**: The time series of the differenced closing prices is stationary, as indicated by both the visual inspection of rolling statistics and the results of the Dickey-Fuller test. This makes the differenced time series suitable for further time series modeling and forecasting.



In [74]:
stock_data['Close']

Date
2019-05-21     41.015999
2019-05-22     38.546001
2019-05-23     39.098000
2019-05-24     38.125999
2019-05-28     37.740002
                 ...    
2022-05-16    724.369995
2022-05-17    761.609985
2022-05-18    709.809998
2022-05-19    709.419983
2022-05-20    663.900024
Name: Close, Length: 758, dtype: float64

In [75]:
train_df = stock_data['Close'][0:-60]
test_df = stock_data['Close'][-60:]

# Function to ensure input is DataFrame
def ensure_dataframe(df):
    if isinstance(df, pd.Series):
        df = df.to_frame(name='Stationary_Close')
    return df

# Function to ensure index is datetime
def ensure_datetime_index(df):
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df.index = pd.to_datetime(df.index, errors='coerce')  # Coerce errors to NaT if any
    return df

# Plotting function
def plot_time_series(train_data, test_data):
    # Ensure each dataset is a DataFrame
    train_data = ensure_dataframe(train_data)
    test_data = ensure_dataframe(test_data)

    # Ensure the index is in datetime format
    train_data = ensure_datetime_index(train_data)
    test_data = ensure_datetime_index(test_data)

    # Create figure
    fig = go.Figure()

    # Plot the training data
    fig.add_trace(go.Scatter(
        x=train_data.index,
        y=train_data['Stationary_Close'],
        mode='lines',
        name='Training Data',
        line=dict(color='blue')
    ))

    # Plot the test data
    fig.add_trace(go.Scatter(
        x=test_data.index,
        y=test_data['Stationary_Close'],
        mode='lines',
        name='Test Data',
        line=dict(color='orange')
    ))

    # Update layout
    fig.update_layout(
        title='Training and Test Data',
        xaxis_title='Time',
        yaxis_title='Value',
        template='plotly_dark',
        legend=dict(x=0.01, y=0.99, traceorder='normal')
    )

    # Show the figure
    fig.show()

# Example usage
plot_time_series(df_train, df_test)

## Training and Test Data Visualization

The provided code segment is designed to visualize the training and test datasets of stock closing prices. 



1. **Splitting Data**:
   - `train_df`: Contains the stock closing prices excluding the last 60 entries.
   - `test_df`: Contains the last 60 entries of the stock closing prices for testing.

2. **Plot**: Plotted taring and test data visualization graph .


In [76]:
history=[x for x in train_df]

In [77]:
def train_arima_model(X, y, arima_order):
    # prepare training dataset
    # make predictions list
    history = [x for x in X]
    predictions = list()
    for t in range(len(y)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(y[t])
    # calculate out of sample error
    rmse = np.sqrt(mean_squared_error(y, predictions))
    return rmse

## ARIMA Model Training and Evaluation

The provided functions are designed to train and evaluate ARIMA (AutoRegressive Integrated Moving Average) models on a time series dataset by testing different combinations of `p`, `d`, and `q` values. The goal is to find the optimal ARIMA order that minimizes the RMSE (Root Mean Squared Error).

### 1. `train_arima_model(X, y, arima_order)`

This function trains an ARIMA model on a given time series dataset and returns the RMSE based on out-of-sample predictions.

#### Key Steps:
- **Input**:
  - `X`: The training dataset (independent variables).
  - `y`: The testing dataset (dependent variables).
  - `arima_order`: A tuple specifying the ARIMA order `(p, d, q)`.

- **Process**:
  - Initializes `history` with the training data.
  - Loops through the test dataset and fits an ARIMA model on the `history`.
  - Makes predictions using the fitted model and appends the actual values from `y` to the `history` to simulate one-step-ahead forecasting.
  - Calculates the RMSE using the actual `y` values and the predictions.

- **Return**:
  - Returns the RMSE (Root Mean Squared Error), which quantifies the prediction error.

In [78]:
def evaluate_models(train, test, p_values, d_values, q_values):
    train = train.astype('float32')
    best_score, best_cfg = float("inf"), None
    
    # Loop over all combinations of p, d, q
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    rmse = train_arima_model(train, test, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print(f'ARIMA{order} MAPE={rmse:.3f}')
                except :
                    continue
    
    print(f'Best ARIMA{best_cfg} RMSE={best_score:.3f}')
    

## 2. `evaluate_models(train, test, p_values, d_values, q_values)`

This function evaluates different ARIMA configurations by testing various combinations of `p`, `d`, and `q` values to find the optimal ARIMA model with the lowest RMSE (Root Mean Squared Error).

### Key Steps:

#### Input:
- **train**: The training dataset.
- **test**: The test dataset.
- **p_values**, **d_values**, **q_values**: Ranges of values for the ARIMA order parameters (p, d, q).

#### Process:
1. Loops through all combinations of `p`, `d`, and `q`.
2. For each combination, it calls the `train_arima_model()` function to train and evaluate an ARIMA model.
3. Tracks the best ARIMA configuration, i.e., the one with the lowest RMSE.
4. Prints the RMSE for each configuration and the best model found.

#### Return:
- Outputs the **best ARIMA configuration** and its corresponding **RMSE** (the model with the lowest RMSE score).

```python

In [79]:
# evaluate parameters
import warnings
warnings.filterwarnings('ignore')
p_values = range(0, 3)
d_values = range(0, 3)
q_values = range(0, 3)
evaluate_models(train_df, test_df, p_values, d_values, q_values)

ARIMA(0, 0, 0) MAPE=457.414
ARIMA(0, 0, 1) MAPE=241.164
ARIMA(0, 0, 2) MAPE=170.901
ARIMA(0, 1, 0) MAPE=39.516
ARIMA(0, 1, 1) MAPE=39.482
ARIMA(0, 1, 2) MAPE=39.617
ARIMA(0, 2, 0) MAPE=57.835
ARIMA(0, 2, 1) MAPE=39.611
ARIMA(0, 2, 2) MAPE=39.580
ARIMA(1, 0, 0) MAPE=39.477
ARIMA(1, 0, 1) MAPE=39.449
ARIMA(1, 0, 2) MAPE=39.584
ARIMA(1, 1, 0) MAPE=39.475
ARIMA(1, 1, 1) MAPE=39.555
ARIMA(1, 1, 2) MAPE=39.935
ARIMA(1, 2, 0) MAPE=46.184
ARIMA(1, 2, 1) MAPE=39.573
ARIMA(1, 2, 2) MAPE=39.731
ARIMA(2, 0, 0) MAPE=39.440
ARIMA(2, 0, 1) MAPE=39.494
ARIMA(2, 0, 2) MAPE=39.581
ARIMA(2, 1, 0) MAPE=39.635
ARIMA(2, 1, 1) MAPE=39.759
ARIMA(2, 1, 2) MAPE=39.631
ARIMA(2, 2, 0) MAPE=45.781
ARIMA(2, 2, 1) MAPE=39.739
ARIMA(2, 2, 2) MAPE=39.732
Best ARIMA(2, 0, 0) RMSE=39.440


## Parameter Evaluation for ARIMA Model

This code snippet is used to evaluate various combinations of ARIMA parameters (`p`, `d`, `q`) to identify the optimal configuration that minimizes the RMSE (Root Mean Squared Error) between the predicted and actual values in the test dataset.

### Key Steps:

1. **Parameter Ranges**:
   - `p_values`: A range of values for the autoregressive (`AR`) term, from 0 to 2.
   - `d_values`: A range of values for the differencing term (`I`), from 0 to 2.
   - `q_values`: A range of values for the moving average (`MA`) term, from 0 to 2.

2. **Suppressing Warnings**:
   - `warnings.filterwarnings('ignore')`: This line ensures that any warnings generated during model training (such as convergence warnings) are suppressed, providing a cleaner output.

3. **Model Evaluation**:
   - `evaluate_models(train_df, test_df, p_values, d_values, q_values)`: The function `evaluate_models` iterates through all possible combinations of `p`, `d`, and `q` values, training an ARIMA model for each combination.
   - The function computes the RMSE for each configuration and prints it. Additionally, it keeps track of the best performing ARIMA configuration (the one with the lowest RMSE).

### Code:

In [80]:
history = [x for x in train_df]
predictions = list()
conf_list = list()
for t in range(len(test_df)):
    model = ARIMA(history,order=(2,0,0))
    model_fit = model.fit()
    fc = model_fit.forecast(alpha = 0.05)
    predictions.append(fc)
    history.append(test_df[t])
print('RMSE of ARIMA Model:', np.sqrt(mean_squared_error(test_df, predictions)))

RMSE of ARIMA Model: 39.43993348000926


## Best ARIMA Model Forecasting with Confidence Intervals

This code segment demonstrates how to forecast future values using the ARIMA model, iterating through the test dataset while updating the model with each new observation. Additionally, it calculates the RMSE (Root Mean Squared Error) to evaluate the accuracy of the predictions.

### Key Steps:

1. **Initialization**:
   - `history`: A list that stores the historical data, initially containing the training dataset (`train_df`).
   - `predictions`: An empty list to store the forecasted values.
   - `conf_list`: An empty list intended to store confidence intervals (though unused in this snippet).

2. **Forecasting Loop**:
   - For each time step `t` in the test dataset (`test_df`):
     - **Model Fitting**: The ARIMA model is instantiated with the optimized specified order `(p=2, d=0, q=0)`. It is fitted to the `history` data using the `fit()` method.
     - **Forecast**: The model generates a one-step forecast with a 95% confidence interval (`alpha = 0.05`).
     - The forecasted value is appended to the `predictions` list.
     - The observed value at time step `t` is then appended to the `history`, updating the model for the next iteration.

3. **RMSE Calculation**:
   - After making all predictions, the RMSE (Root Mean Squared Error) is computed between the actual values (`test_df`) and the predicted values (`predictions`).



In [109]:
print('MAPE of ARIMA Model:', mean_absolute_percentage_error(test_df[0:10], predictions[0:10]))

MAPE of ARIMA Model: 0.0284602190480936


In [110]:
# For Manual Forecasting
'''import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

# Fit the ARIMA model to the training data
model = ARIMA(train_df, order=(2, 0, 0))
model_fit = model.fit()

# Forecast the next 100 periods
n_periods = 100
forecast = model_fit.forecast(alpha = 0.05,steps=n_periods)

# Store forecasted values in a variable
forecast_values = forecast

# Print the stored forecast values
print(f'Forecasted values for the next {n_periods} periods: {forecast_values}')

# Create a figure
fig = go.Figure()

# Add historical data trace
fig.add_trace(go.Scatter(x=np.arange(len(train_df)), y=train_df,
                         mode='lines', name='Historical Data',
                         line=dict(color='blue', width=2)))

# Add forecast trace
fig.add_trace(go.Scatter(x=np.arange(len(train_df), len(train_df) + n_periods),
                         y=forecast_values, mode='lines', name='Forecasted Values',
                         line=dict(color='red', width=2)))

# Customize layout
fig.update_layout(
    title='ARIMA Forecast for the Next 100 Periods',
    xaxis_title='Time',
    yaxis_title='Value',
    title_x=0.5,
    template='plotly_dark',  # Optional dark theme
    width=1000,  # Adjust width of the figure
    height=600,  # Adjust height of the figure
    showlegend=True
)

# Show figure
fig.show()
'''


"import plotly.graph_objects as go\nfrom statsmodels.tsa.arima.model import ARIMA\nimport numpy as np\n\n# Fit the ARIMA model to the training data\nmodel = ARIMA(train_df, order=(2, 0, 0))\nmodel_fit = model.fit()\n\n# Forecast the next 100 periods\nn_periods = 100\nforecast = model_fit.forecast(alpha = 0.05,steps=n_periods)\n\n# Store forecasted values in a variable\nforecast_values = forecast\n\n# Print the stored forecast values\nprint(f'Forecasted values for the next {n_periods} periods: {forecast_values}')\n\n# Create a figure\nfig = go.Figure()\n\n# Add historical data trace\nfig.add_trace(go.Scatter(x=np.arange(len(train_df)), y=train_df,\n                         mode='lines', name='Historical Data',\n                         line=dict(color='blue', width=2)))\n\n# Add forecast trace\nfig.add_trace(go.Scatter(x=np.arange(len(train_df), len(train_df) + n_periods),\n                         y=forecast_values, mode='lines', name='Forecasted Values',\n                         line

In [108]:
print('MAPE of ARIMA Model:', mean_absolute_percentage_error(test_df[0:10], forecast_values[0:10]))

MAPE of ARIMA Model: 0.054412495136697124


In [115]:

predictions = np.array(predictions).flatten()

# Create a Plotly figure
fig = go.Figure()

# Add true test data trace
fig.add_trace(go.Scatter(
    x=np.arange(len(test_df)),
    y=test_df,
    mode='lines',
    name='True Test Close Value',
    line=dict(color='blue', width=5)
))

# Add predictions trace
fig.add_trace(go.Scatter(
    x=np.arange(len(predictions)),
    y=predictions,
    mode='lines',
    name='Predictions on Test Data',
    line=dict(color='red', width=5)
))

# Customize layout
fig.update_layout(
    title='True Test Values vs Predictions',
    xaxis_title='Index',
    yaxis_title='Close Value',
    title_x=0.5,
    template='plotly_dark',  # Optional dark theme
    width=1000,  # Adjust width of the figure
    height=600,  # Adjust height of the figure
    showlegend=True,
    legend=dict(font=dict(size=20), bordercolor='black', borderwidth=2)  # Removed bgcolor
)

# Show figure
fig.show()


1. **Data Preparation**:
   - The predictions are converted to a flat array using `predictions = np.array(predictions).flatten()` to ensure the correct format for plotting.

2. **Plot Creation**:
   - A `Plotly` figure is created using `go.Figure()` to hold the plot.
   
3. **True Test Data Trace**:
   - The true test data values (`test_df`) are plotted as a blue line representing the actual stock closing prices for the test dataset.

4. **Predictions Trace**:
   - The predicted values from the ARIMA model are plotted as a red line, representing the model's forecasts on the test data.

5. **Layout Customization**:
   - **Title and Axis Labels**: The plot is given a title ("True Test Values vs Predictions") and labeled axes ("Index" for x and "Close Value" for y).
   - **Dark Theme**: A dark theme (`template='plotly_dark'`) is applied to the plot for visual appeal.
   - **Figure Size**: The figure's width and height are adjusted to 1000px and 600px, respectively, for better clarity.
   - **Legend Customization**: The legend is styled to have a larger font size and a black border.


In [143]:

fc_series = pd.Series(predictions, index=test_df.index)
# Create the Plotly figure
fig = go.Figure()

# Plot Training data
fig.add_trace(go.Scatter(
    x=np.arange(len(train_df)),  # Use the index as x
    y=train_df,
    mode='lines',
    name='Training',
    line=dict(color='blue')
))

# Plot Test data
fig.add_trace(go.Scatter(
    x=np.arange(len(train_df), len(train_df) + len(test_df)),  # Index for test data starts after train_data
    y=test_df,
    mode='lines',
    name='Test',
    line=dict(color='red', width=3)
))

# Plot Forecast data
fig.add_trace(go.Scatter(
    x=np.arange(len(train_df), len(train_df) + len(fc_series)),  # Index for forecast data starts after train_data
    y=fc_series,
    mode='lines',
    name='Forecast',
    line=dict(color='orange')
))

# Customize layout
fig.update_layout(
    title='Forecast vs Actuals on Test Data',
    title_x=0.5,
    xaxis_title='Index',
    yaxis_title='Value',
    legend=dict(font=dict(size=8), bordercolor='black', borderwidth=2),
    width=1200,  # Width of the figure
    height=600,  # Height of the figure
    template='plotly_dark'  # Optional dark theme
)

# Show figure
fig.show()


## Visualization: Forecast vs Actuals on Test Data

This code generates an interactive Plotly figure to compare the forecasted values from the ARIMA model against the actual test data. It also visualizes the training data for better context.

### Key Steps:

1. **Forecast Data**:
   - The predicted values (`predictions`) are converted to a pandas Series (`fc_series`) and aligned with the index of the test data (`test_df.index`).

2. **Plot Creation**:
   - A `Plotly` figure is created to display three separate traces: 
     - **Training Data**: Plotted as a blue line representing the historical stock prices used for model training.
     - **Test Data**: Plotted as a red line showing the actual stock prices in the test dataset.
     - **Forecast Data**: Plotted as an orange line showing the predicted stock prices made by the ARIMA model on the test data.

3. **Indexing**:
   - The x-axis is set using `np.arange` to ensure correct positioning of the test and forecast data relative to the training data.

4. **Layout Customization**:
   - **Title**: The plot is titled "Forecast vs Actuals on Test Data" and centered using `title_x=0.5`.
   - **Axes Labels**: The x-axis is labeled "Index" and the y-axis is labeled "Value".
   - **Legend**: The legend is customized with a smaller font and a black border for better readability.
   - **Size**: The figure is set to a width of 1200px and height of 600px for clarity.
   - **Theme**: A dark theme (`template='plotly_dark'`) is applied to the figure.


In [134]:
print('MAPE of ARIMA Model:', mean_absolute_percentage_error(test_df[0:16], predictions[0:16]))

MAPE of ARIMA Model: 0.034064729100652294


In [153]:

predictions = np.array(predictions).flatten()

# Create a Plotly figure
fig = go.Figure()

# Add true test data trace
fig.add_trace(go.Scatter(
    x=np.arange(len(test_df[0:16])),
    y=test_df,
    mode='lines',
    name='True Test Close Value',
    line=dict(color='blue', width=5)
))

# Add predictions trace
fig.add_trace(go.Scatter(
    x=np.arange(len(predictions[0:16])),
    y=predictions,
    mode='lines',
    name='Predictions on Test Data',
    line=dict(color='red', width=5)
))

# Customize layout
fig.update_layout(
    title='True Test Values vs next 16 Predictions',
    xaxis_title='Index',
    yaxis_title='Close Value',
    title_x=0.5,
    template='plotly_dark',  # Optional dark theme
    width=1000,  # Adjust width of the figure
    height=600,  # Adjust height of the figure
    showlegend=True,
    legend=dict(font=dict(size=20), bordercolor='black', borderwidth=2)  # Removed bgcolor
)

# Show figure
fig.show()


## Plotting: True Test Values vs Predictions

This code generates a Plotly line graph to visualize the first 16 predicted values from an ARIMA model alongside the actual test data for comparison.

### Key Steps:

1. **Data Preparation**:
   - The predictions (`predictions`) are converted to a NumPy array and flattened to ensure compatibility for plotting.
   - Only the first 16 values from both `test_df` (the true test data) and `predictions` (the model's predicted values) are considered for visualization.

2. **Plot Creation**:
   - A `Plotly` figure is created with two traces:
     - **True Test Close Values**: Plotted as a blue line, representing the actual stock closing prices from the test dataset.
     - **Predicted Values**: Plotted as a red line, representing the predicted stock closing prices from the ARIMA model.

3. **Layout Customization**:
   - **Title**: The plot is titled "True Test Values vs Predictions," and the title is centered.
   - **Axes Labels**: The x-axis is labeled "Index," and the y-axis is labeled "Close Value."
   - **Legend**: The legend is customized with a larger font size and a border.
   - **Size**: The figure's width is set to 1000px and the height to 600px, ensuring a clear and readable output.
   - **Theme**: The plot uses the `plotly_dark` template for a visually appealing dark theme.

4. **Display**:
   - The `fig.show()` method is called to display the interactive visualization.


In [136]:
mape=mean_absolute_percentage_error(test_df[0:16], predictions[0:16])
accuracy = 1 - mape

print(f'Mean Absolute Percentage Error (MAPE): {mape * 100:.2f}%')
print(f'Accuracy: {accuracy * 100:.2f}%')

Mean Absolute Percentage Error (MAPE): 3.41%
Accuracy: 96.59%


### Model Evaluation: MAPE and Accuracy

This code calculates the **Mean Absolute Percentage Error (MAPE)** and **Accuracy** of the ARIMA model's predictions over the first 16 values of the test dataset.

#### Key Steps:

1. **MAPE Calculation**:
   - The **Mean Absolute Percentage Error (MAPE)** is calculated by comparing the first 16 values from both the `test_df` (actual values) and `predictions`. 
   - MAPE is a commonly used metric to evaluate model performance, measuring the average percentage error between the true values and the predictions.
   
   \[
   \text{MAPE} = \frac{1}{n} \sum \left|\frac{y_i - \hat{y}_i}{y_i}\right| \times 100
   \]
   A lower MAPE indicates better model accuracy.

2. **Accuracy Calculation**:
   - The **Accuracy** is derived as \( 1 - \text{MAPE} \), giving a percentage that reflects how accurate the model's predictions are.
   
3. **Results**:
   - **MAPE**: The model's MAPE is calculated to be **3.41%**, meaning the model's predictions deviate by an average of 3.41% from the actual values.
   - **Accuracy**: The accuracy of the model is **96.59%**, indicating that the model predicts the test data with high accuracy.

